# AI R&D Intern Assessment: RAG Pipeline for Financial Data QA

## Step 1: Basic RAG Pipeline

**Objective:** Build a simple RAG pipeline for factual QA from Meta's Q1 2024 financial report.

**Deliverables:**
- Source code (this notebook)
- Brief report (see below)
- Sample outputs for test queries

---

## Brief Report

**Approach & Tools:**
- PDF text extraction: PyMuPDF (fitz)
- Chunking: Simple text splitter
- Embedding: Sentence Transformers (all-MiniLM-L6-v2)
- Vector DB: FAISS
- LLM: Open-source LLM (e.g., Llama.cpp, or use HuggingFace pipeline with a small model for demo)

**Challenges & Results:**
- Table extraction and hybrid retrieval are not included in Step 1.
- The pipeline retrieves relevant chunks and generates answers for factual queries.

---

## 1. Preprocessing: Extract and Clean Text from PDF

In [70]:
import fitz
import re

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''
    for page in doc:
        text += page.get_text() + '\n'
    # Basic cleaning
    text = re.sub(r'\n+', '\n', text)
    return text

pdf_path = 'Meta’s Q1 2024 Financial Report.pdf'
raw_text = extract_text_from_pdf(pdf_path)
print(raw_text[:5000]) 

Meta Reports First Quarter 2024 Results
MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter 
ended March 31, 2024.
"It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is 
another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making 
steady progress building the metaverse as well."
First Quarter 2024 Financial Highlights
Three Months Ended March 31,
% Change
In millions, except percentages and per share amounts
2024
2023
Revenue
$ 
36,455 
$ 
28,645 
 27 %
Costs and expenses
 
22,637 
 
21,418 
 6 %
Income from operations
$ 
13,818 
$ 
7,227 
 91 %
Operating margin
 38 %
 25 %
Provision for income taxes
$ 
1,814 
$ 
1,598 
 14 %
Effective tax rate
 13 %
 22 %
Net income
$ 
12,369 
$ 
5,709 
 117 %
Diluted earnings per share (EPS)
$ 
4.71 
$ 
2.20 
 114 %
First Quarter 2024 Operational and

## 2. Chunking & Embedding
- Split text into manageable chunks
- Generate embeddings using Sentence Transformers

In [131]:
import sys
import subprocess

def install(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

install('sentence_transformers')
install('numpy')

from sentence_transformers import SentenceTransformer
import numpy as np

def chunk_text(text, chunk_size=500, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i+chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

chunks = chunk_text(raw_text)
print(f'Number of chunks: {len(chunks)}')
print(len(raw_text.split()))
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(chunks, show_progress_bar=True)

Number of chunks: 6
2530


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.50it/s]


## 3. Retrieval: Vector Similarity Search
- Use FAISS to retrieve top-3 relevant chunks for a query

In [132]:
# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

def retrieve(query, k=3):
    query_emb = embedder.encode([query])
    D, I = index.search(np.array(query_emb).astype('float32'), k)
    return [chunks[i] for i in I[0]]

## 4. Generation: Answer Queries with LLM
- Use an open-source LLM (e.g., HuggingFace pipeline)
- Prompt: Based on the following context: {context} Answer the query: {query}

In [133]:
def generate_fact_answer(context, query):
    """Extract the most relevant sentence(s) from the context for direct fact-based queries."""
    import re
    context_lower = context.lower()
    query_lower = query.lower()
    sentences = re.split(r'(?<=[.!?])\s+', context)
    keywords = [w for w in re.findall(r'\w+', query_lower) if len(w) > 2]
    best_sentences = []
    for sent in sentences:
        sent_lower = sent.lower()
        if all(k in sent_lower for k in keywords[:2]):  
            best_sentences.append(sent)
    if not best_sentences:
        for sent in sentences:
            if 'revenue' in sent.lower() or 'highlight' in sent.lower():
                best_sentences.append(sent)
    if not best_sentences:
        best_sentences = sentences[:2]
    return ' '.join(best_sentences)

## 5. Test Queries & Sample Outputs

In [134]:
test_queries = [
    "What was Meta’s revenue in Q1 2024?",
    "What were the key financial highlights for Meta in Q1 2024?"
 ]

import pandas as pd
results = []
for query in test_queries:
    top_chunks = retrieve(query)
    seen = set()
    deduped_chunks = []
    for chunk in top_chunks:
        if chunk not in seen:
            deduped_chunks.append(chunk)
            seen.add(chunk)
    context = '\n'.join(deduped_chunks)
    answer = generate_fact_answer(context, query)
    import re
    sentences = re.split(r'(?<=[.!?])\s+', context)
    snippet = ''
    for sent in sentences:
        if len(sent.strip()) > 20:
            snippet = sent.strip()
            break
    if not snippet:
        snippet = context[:200] + ('...' if len(context) > 200 else '')
    results.append({'Query': query, 'Answer': answer.strip(), 'Context Snippet': snippet})

pd.set_option('display.max_colwidth', 300)
display(pd.DataFrame(results))

,Query,Answer,Context Snippet
0,What was Meta’s revenue in Q1 2024?,"We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well."" First Quarter 2024 Financial Highlights Three Months Ended March 31, % Change In millions, except percentages and per share amounts 2024 2023 Revenue $ 36,455 $ 28,645 27 % Costs a...","Meta Reports First Quarter 2024 Results MENLO PARK, Calif."
1,What were the key financial highlights for Meta in Q1 2024?,"Apps like Messenger, Instagram, and WhatsApp further empowered billions around the world.","Meta Reports First Quarter 2024 Results MENLO PARK, Calif."


## Step 2: Structured Data Integration
- Extract tables from PDF into DataFrames
- Hybrid retrieval: combine vector search (text) and keyword/SQL-like search (structured)
- Updated prompt includes both text and structured data


In [162]:
import pandas as pd
import re
import warnings

def is_valid_table(df):
    if df.shape[1] < 2 or df.shape[0] < 2:
        return False
    header = list(df.columns)
    empty_header_count = sum([(str(col).strip() == '' or str(col).strip().lower() == 'none') for col in header])
    if empty_header_count > len(header) // 2:
        return False
    return True

def clean_table(df):
    df = df.loc[:, ~(df.apply(lambda col: all(str(x).strip() == '' or str(x).strip().lower() == 'none' for x in col)))]
    df = df.loc[~df.apply(lambda row: all(str(x).strip() == '' or str(x).strip().lower() == 'none' for x in row), axis=1)]
    if hasattr(df, 'map'):
        df = df.map(lambda x: str(x).strip() if x is not None else '')
    else:
        df = df.applymap(lambda x: str(x).strip() if x is not None else '')
    return df

def extract_multiline_heading(lines, table_start_idx):
    heading_lines = []
    idx = table_start_idx - 1
    while idx >= 0 and lines[idx].strip():
        heading_lines.insert(0, lines[idx].strip())
        idx -= 1
    return '\n'.join(heading_lines).strip()

tables = []
table_titles = []

# --- PDFPLUMBER EXTRACTION ---
import pdfplumber
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages):
        text = page.extract_text() or ''
        lines = text.split('\n') if text else []
        tables_on_page = page.extract_tables()
        for t_idx, table in enumerate(tables_on_page):
            first_row = table[0] if table and len(table) > 0 else []
            first_row_str = [str(cell) if cell is not None else '' for cell in first_row]
            table_start_idx = -1
            for i, line in enumerate(lines):
                if ' '.join(first_row_str).replace(' ', '') in line.replace(' ', ''):
                    table_start_idx = i
                    break
            heading = ''
            if table_start_idx > 0:
                heading = extract_multiline_heading(lines, table_start_idx)
            if not heading:
                heading = f"Table on page {page_num+1}, #{t_idx+1}"
            df = pd.DataFrame(table[1:], columns=table[0]) if table and len(table) > 1 else pd.DataFrame(table)
            df = clean_table(df)
            if is_valid_table(df):
                tables.append(df)
                table_titles.append(heading)

# --- CAMELOT EXTRACTION ---
try:
    import camelot
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning, module='camelot')
        camelot_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
        for i, t in enumerate(camelot_tables):
            df = t.df
            df = clean_table(df)
            if is_valid_table(df):
                is_duplicate = any(df.equals(existing) for existing in tables)
                if not is_duplicate:
                    tables.append(df)
                    table_titles.append(f"Camelot Table {i+1}")
except Exception as e:
    pass

from IPython.display import display, Markdown, display_markdown
if tables:
    display(Markdown(f'**Total tables detected: {len(tables)}**'))
    for idx, (title, df) in enumerate(zip(table_titles, tables), 1):
        display(Markdown(f'### {title.replace(chr(10), "<br>")}'))
        table_str = df.to_string(index=False, col_space=0, justify='left')
        display_markdown(f'<pre style="text-align:left">{table_str}</pre>', raw=True)
else:
    display(Markdown('**No tables found in the PDF.**'))

**Total tables detected: 10**

### META PLATFORMS, INC.<br>CONDENSED CONSOLIDATED STATEMENTS OF INCOME<br>(In millions, except per share amounts)<br>(Unaudited)<br>Three Months Ended March 31,<br>2024 2023

<pre style="text-align:left">Revenue                                                     $ 36,455 $ 28,645
                                        Costs and expenses:                  
                                            Cost of revenue      6,640 6,108 
                                   Research and development      9,978 9,381 
                                        Marketing and sales      2,564 3,044 
                                 General and administrative      3,455 2,885 
                                   Total costs and expenses    22,637 21,418 
                                                                13,818 7,227 
                             Interest and other income, net           365 80 
                   Income before provision for income taxes     14,183 7,307 
                                 Provision for income taxes      1,814 1,598 
                                                 Net income $ 12,369 $ 5,709 
                                        Earnings per share:                  
                                                      Basic    $ 4.86 $ 2.21 
                                                    Diluted    $ 4.71 $ 2.20 
Weighted-average shares used to compute earnings per share:                  
                                                      Basic      2,545 2,587 
                                                    Diluted      2,625 2,596 </pre>

### Segment Results<br>We report our financial results for our two reportable segments: Family of Apps (FoA) and Reality Labs (RL). FoA includes<br>Facebook, Instagram, Messenger, WhatsApp, and other services. RL includes our virtual, augmented, and mixed reality related<br>consumer hardware, software, and content.<br>The following table presents our segment information of revenue and income (loss) from operations:<br>Segment Information<br>(In millions)<br>(Unaudited)<br>Three Months Ended March 31,<br>2024 2023

<pre style="text-align:left">Revenue:                                      None    
                         Advertising $ 35,635 $ 28,101
                        Other revenue 380 205         
                 Family of Apps 36,015 28,306         
                         Reality Labs 440 339         
              Total revenue $ 36,455 $ 28,645         
               Income (loss) from operations:         
             Family of Apps $ 17,664 $ 11,219         
                 Reality Labs (3,846) (3,992)         
Total income from operations $ 13,818 $ 7,227         </pre>

...

<pre style="text-align:left">0                                                                           1                                               2    3                            4 5    6    
                                                                                                       META PLATFORMS, INC.                                               
                                                                            CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS                                               
                                                              (In millions)                                                                                               
                                                                (Unaudited)                                                                                               
                                                                                                                                 Three Months Ended March 31,             
                                                                                                                            2024                                2023      
                                                Supplemental cash flow data                                                                                               
                                            Cash paid for income taxes, net                                               $                               630 $        405
                         Cash paid for interest, net of amounts capitalized                                               $                               121 $        182
                               Non-cash investing and financing activities:                                                                                               
        Property and equipment in accounts payable and accrued expenses and                                                                                               
                                                  other current liabilities                                               $                             4,217 $      4,466
Acquisition of businesses in accrued expenses and other current liabilities                                                                                               
                                                      and other liabilities                                               $                               116 $        263
                                                                          8                                                                                               </pre>

### Camelot Table 12

<pre style="text-align:left">0                              1 2    3                            4 5    6      
                                      Three Months Ended March 31,               
                                 2024                                2023        
                      Revenue:                                                   
                   Advertising $                            35,635 $       28,101
                 Other revenue                                 380            205
                Family of Apps                              36,015         28,306
                  Reality Labs                                 440            339
                 Total revenue $                            36,455 $       28,645
Income (loss) from operations:                                                   
                Family of Apps $                            17,664 $       11,219
                  Reality Labs                             (3,846)        (3,992)
  Total income from operations $                            13,818 $        7,227</pre>

### Camelot Table 13

<pre style="text-align:left">0                                                                             1 2    3                            4 5    6      
                                                                  (Unaudited)                                                   
                                                                                     Three Months Ended March 31,               
                                                                                2024                                2023        
                                                                 GAAP revenue $                            36,455 $       28,645
                     Foreign exchange effect on 2024 revenue using 2023 rates                               (106)               
                                    Revenue excluding foreign exchange effect $                            36,349               
                                         GAAP revenue year-over-year change %                                27 %               
            Revenue excluding foreign exchange effect year-over-year change %                                27 %               
                                                     GAAP advertising revenue $                            35,635 $       28,101
         Foreign exchange effect on 2024 advertising revenue using 2023 rates                               (105)               
                        Advertising revenue excluding foreign exchange effect $                            35,530               
                             GAAP advertising revenue year-over-year change %                                27 %               
Advertising revenue excluding foreign exchange effect year-over-year change %                                26 %               
                                    Net cash provided by operating activities $                            19,246 $       13,998
                                     Purchases of property and equipment, net                             (6,400)        (6,823)
                                         Principal payments on finance leases                               (315)          (264)
                                                               Free cash flow $                            12,531 $        6,911</pre>

In [167]:
try:
    import pdfplumber
except ImportError:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pdfplumber'])
    import pdfplumber

import pandas as pd

def is_valid_table(df):
    if df.shape[1] < 2 or df.shape[0] < 2:
        return False
    header = list(df.columns)
    empty_header_count = sum([(str(col).strip() == '' or str(col).strip().lower() == 'none') for col in header])
    if empty_header_count > len(header) // 2:
        return False
    return True

def clean_table(df):
    df = df.loc[:, ~(df.apply(lambda col: all(str(x).strip() == '' or str(x).strip().lower() == 'none' for x in col)))]
    df = df.loc[~df.apply(lambda row: all(str(x).strip() == '' or str(x).strip().lower() == 'none' for x in row), axis=1)]
    if hasattr(df, 'map'):
        df = df.map(lambda x: str(x).strip() if x is not None else '')
    else:
        df = df.applymap(lambda x: str(x).strip() if x is not None else '')
    return df

pdfplumber_tables = []
pdfplumber_table_names = []
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages):
        lines = page.extract_text().split('\n') if page.extract_text() else []
        tables_on_page = page.extract_tables()
        for t_idx, table in enumerate(tables_on_page):
            table_bbox = None
            if hasattr(page, 'find_tables'):
                found_tables = page.find_tables()
                if t_idx < len(found_tables):
                    table_bbox = found_tables[t_idx].bbox
            table_name = f"Table on page {page_num+1}, #{t_idx+1}"
            if table_bbox and lines:
                y0 = table_bbox[1]
                for l_idx in range(len(lines)-1, -1, -1):
                    if lines[l_idx].strip():
                        table_name = lines[l_idx].strip()
                        break
            df = pd.DataFrame(table[1:], columns=table[0]) if table and len(table) > 1 else pd.DataFrame(table)
            df = clean_table(df)
            if is_valid_table(df):
                pdfplumber_tables.append(df)
                pdfplumber_table_names.append(table_name)
                # Also append to main tables/titles for hybrid retrieval
                if 'tables' in globals():
                    tables.append(df)
                    if 'table_titles' in globals():
                        table_titles.append(table_name)

print(f"pdfplumber extracted {len(pdfplumber_tables)} valid tables.")
from IPython.display import display, Markdown
for i, (df, name) in enumerate(zip(pdfplumber_tables, pdfplumber_table_names)):
    display(Markdown(f'### {name}'))
    display(df)  # Show the full table, not just the head

pdfplumber extracted 2 valid tables.


### 5

,Revenue,"$ 36,455 $ 28,645"
0,Costs and expenses:,
1,Cost of revenue,"6,640 6,108"
2,Research and development,"9,978 9,381"
3,Marketing and sales,"2,564 3,044"
4,General and administrative,"3,455 2,885"
5,Total costs and expenses,"22,637 21,418"
6,,"13,818 7,227"
7,"Interest and other income, net",365 80
8,Income before provision for income taxes,"14,183 7,307"
9,Provision for income taxes,"1,814 1,598"


### 9

,Revenue:,None
0,"Advertising $ 35,635","$ 28,101"
1,Other revenue 380 205,
2,"Family of Apps 36,015 28,306",
3,Reality Labs 440 339,
4,"Total revenue $ 36,455 $ 28,645",
6,Income (loss) from operations:,
7,"Family of Apps $ 17,664 $ 11,219",
8,"Reality Labs (3,846) (3,992)",
9,"Total income from operations $ 13,818 $ 7,227",


In [168]:
import pandas as pd
def hybrid_retrieve(query, k=3):
    text_chunks = retrieve(query, k)
    table_results = []
    if tables:
        for table in tables:
            mask = table.apply(lambda col: col.astype(str).str.contains(query, case=False, na=False))
            if mask.any().any():
                table_results.append(table[mask.any(axis=1)])
    return text_chunks, table_results

test_query2 = "net income"
text_chunks, table_results = hybrid_retrieve(test_query2)

import re
snippet = ''
for chunk in text_chunks:
    sentences = re.split(r'(?<=[.!?])\s+', chunk)
    for sent in sentences:
        if len(sent.strip()) > 20:
            snippet = sent.strip()
            break
    if snippet:
        break
if not snippet and text_chunks:
    snippet = text_chunks[0][:200] + ('...' if len(text_chunks[0]) > 200 else '')

table_str = "No relevant table rows found."
if table_results:
    table_str = table_results[0].head(2).to_string(index=False)

results = [{
    'Query': test_query2,
    'Text Snippet': snippet,
    'Table Row(s)': table_str,
}]
pd.set_option('display.max_colwidth', 300)
display(pd.DataFrame(results))

,Query,Text Snippet,Table Row(s)
0,net income,"and other current liabilities $ 4,217 $ 4,466 Acquisition of businesses in accrued expenses and other current liabilities and other liabilities $ 116 $ 263 8 Segment Results We report our financial results for our two reportable segments: Family of Apps (FoA) and Reality Labs (RL).","Revenue $ 36,455 $ 28,645\nNet income $ 12,369 $ 5,709"


In [179]:
# Step 2: Hybrid Answer Extraction (with explicit comparison and percent change)
test_queries_step2 = [
    "What was Meta’s net income in Q1 2024 compared to Q1 2023?",
    "Summarize Meta’s operating expenses in Q1 2024."
]

def extract_years_from_value(val):
    import re
    nums = re.findall(r'[\d,]+', val)
    nums = [int(n.replace(',', '')) for n in nums]
    if len(nums) >= 2:
        return nums[0], nums[1]
    elif len(nums) == 1:
        return nums[0], None
    else:
        return None, None

def format_comparison(q, y2024, y2023, label):
    if y2024 is not None and y2023 is not None:
        diff = y2024 - y2023
        pct = (diff / y2023 * 100) if y2023 != 0 else None
        if "net income" in q.lower():
            return f"Meta's net income in Q1 2024 was ${y2024:,}, compared to ${y2023:,} in Q1 2023. This is a change of ${diff:,} ({pct:.1f}% {'increase' if diff > 0 else 'decrease'})."
        elif "operating expenses" in q.lower():
            return f"Meta's operating expenses in Q1 2024 were ${y2024:,}, compared to ${y2023:,} in Q1 2023. This is a change of ${diff:,} ({pct:.1f}% {'increase' if diff > 0 else 'decrease'})."
    elif y2024 is not None:
        return f"{label} in Q1 2024: ${y2024:,}."
    else:
        return "Not found."

results = []
for query in test_queries_step2:
    answer = "Not found"
    table_str = "No relevant table rows found."
    for df in tables:
        if list(df.columns)[0].lower().startswith('revenue'):
            if "net income" in query.lower():
                match = df[df[df.columns[0]].str.lower().str.strip() == "net income"]
                if not match.empty:
                    val = match.iloc[0, 1]
                    y2024, y2023 = extract_years_from_value(val)
                    answer = format_comparison(query, y2024, y2023, "Net income")
                    table_str = match.to_string(index=False)
                    break
            elif "operating expenses" in query.lower():
                match = df[df[df.columns[0]].str.lower().str.strip() == "total costs and expenses"]
                if not match.empty:
                    val = match.iloc[0, 1]
                    y2024, y2023 = extract_years_from_value(val)
                    answer = format_comparison(query, y2024, y2023, "Operating expenses")
                    table_str = match.to_string(index=False)
                    break
    results.append({
        'Query': query,
        'Table Row(s)': table_str,
        'Hybrid Answer': answer
    })

import pandas as pd
pd.set_option('display.max_colwidth', 300)
display(pd.DataFrame(results))

,Query,Table Row(s),Hybrid Answer
0,What was Meta’s net income in Q1 2024 compared to Q1 2023?,"Revenue $ 36,455 $ 28,645\nNet income $ 12,369 $ 5,709","Meta's net income in Q1 2024 was $12,369, compared to $5,709 in Q1 2023. This is a change of $6,660 (116.7% increase)."
1,Summarize Meta’s operating expenses in Q1 2024.,"Revenue $ 36,455 $ 28,645\nTotal costs and expenses 22,637 21,418","Meta's operating expenses in Q1 2024 were $22,637, compared to $21,418 in Q1 2023. This is a change of $1,219 (5.7% increase)."


In [ ]:
from IPython.display import display, Markdown
if 'tables' in globals() and tables:
    display(Markdown(f'**Total tables detected: {len(tables)}**'))
    for idx, (title, df) in enumerate(zip(table_titles, tables), 1):
        display(Markdown(f'### Table {idx}: {title}'))
        display(Markdown(f'**Columns:** {list(df.columns)}'))
        display(df)
else:
    display(Markdown('**No tables found in the PDF.**'))

**Total tables detected: 16**

### Table 1: META PLATFORMS, INC.
CONDENSED CONSOLIDATED STATEMENTS OF INCOME
(In millions, except per share amounts)
(Unaudited)
Three Months Ended March 31,
2024 2023

**Columns:** ['Revenue', '$ 36,455 $ 28,645']

,Revenue,"$ 36,455 $ 28,645"
0,Costs and expenses:,
1,Cost of revenue,"6,640 6,108"
2,Research and development,"9,978 9,381"
3,Marketing and sales,"2,564 3,044"
4,General and administrative,"3,455 2,885"
5,Total costs and expenses,"22,637 21,418"
6,,"13,818 7,227"
7,"Interest and other income, net",365 80
8,Income before provision for income taxes,"14,183 7,307"
9,Provision for income taxes,"1,814 1,598"


### Table 2: Segment Results
We report our financial results for our two reportable segments: Family of Apps (FoA) and Reality Labs (RL). FoA includes
Facebook, Instagram, Messenger, WhatsApp, and other services. RL includes our virtual, augmented, and mixed reality related
consumer hardware, software, and content.
The following table presents our segment information of revenue and income (loss) from operations:
Segment Information
(In millions)
(Unaudited)
Three Months Ended March 31,
2024 2023

...

**Columns:** ['Revenue', '$ 36,455 $ 28,645']

,Revenue,"$ 36,455 $ 28,645"
0,Costs and expenses:,
1,Cost of revenue,"6,640 6,108"
2,Research and development,"9,978 9,381"
3,Marketing and sales,"2,564 3,044"
4,General and administrative,"3,455 2,885"
5,Total costs and expenses,"22,637 21,418"
6,,"13,818 7,227"
7,"Interest and other income, net",365 80
8,Income before provision for income taxes,"14,183 7,307"
9,Provision for income taxes,"1,814 1,598"


### Table 16: 9

**Columns:** ['Revenue:', None]

,Revenue:,None
0,"Advertising $ 35,635","$ 28,101"
1,Other revenue 380 205,
2,"Family of Apps 36,015 28,306",
3,Reality Labs 440 339,
4,"Total revenue $ 36,455 $ 28,645",
6,Income (loss) from operations:,
7,"Family of Apps $ 17,664 $ 11,219",
8,"Reality Labs (3,846) (3,992)",
9,"Total income from operations $ 13,818 $ 7,227",


## Step 3: Query Optimization & Advanced RAG
- Query rewriting/optimization
- Advanced retrieval (reranking, chunk size experiments)
- Evaluation framework (Precision@k, Recall@k, MRR, BLEU, ROUGE)
- Performance analysis and ablation study


In [180]:
# Step 3: Query Optimization and Advanced Retrieval
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def optimize_query(query):
    """Rewrite or optimize the query using LLM or rules. For demo, returns original query."""
    return query

def rerank(query, candidate_chunks):
    """Rerank candidate chunks using a cross-encoder model."""
    tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
    model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
    scores = []
    for chunk in candidate_chunks:
        inputs = tokenizer(query, chunk, return_tensors='pt', truncation=True)
        with torch.no_grad():
            score = model(**inputs).logits[0].item()
        scores.append(score)
    reranked = [x for _, x in sorted(zip(scores, candidate_chunks), reverse=True)]
    return reranked

# Example usage
query = "Summarize Meta’s operating expenses in Q1 2024."
optimized_query = optimize_query(query)
candidate_chunks = retrieve(optimized_query, k=5)
reranked_chunks = rerank(optimized_query, candidate_chunks)
print("Top reranked chunk:\n", reranked_chunks[0])

Top reranked chunk:
 Meta Reports First Quarter 2024 Results MENLO PARK, Calif. – April 24, 2024 – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the quarter ended March 31, 2024. "It's been a good start to the year," said Mark Zuckerberg, Meta founder and CEO. "The new version of Meta AI with Llama 3 is another step towards building the world's leading AI. We're seeing healthy growth across our apps and we continue making steady progress building the metaverse as well." First Quarter 2024 Financial Highlights Three Months Ended March 31, % Change In millions, except percentages and per share amounts 2024 2023 Revenue $ 36,455 $ 28,645 27 % Costs and expenses 22,637 21,418 6 % Income from operations $ 13,818 $ 7,227 91 % Operating margin 38 % 25 % Provision for income taxes $ 1,814 $ 1,598 14 % Effective tax rate 13 % 22 % Net income $ 12,369 $ 5,709 117 % Diluted earnings per share (EPS) $ 4.71 $ 2.20 114 % First Quarter 2024 Operational and Other Financial H

In [79]:
# Evaluation framework (simplified demo)
def precision_at_k(relevant, retrieved, k):
    return len(set(relevant) & set(retrieved[:k])) / k

def recall_at_k(relevant, retrieved, k):
    return len(set(relevant) & set(retrieved[:k])) / len(relevant)

def mrr(relevant, retrieved):
    for i, doc in enumerate(retrieved):
        if doc in relevant:
            return 1 / (i + 1)
    return 0

# BLEU/ROUGE can be computed with nltk/rouge-score if needed
# Placeholder for ablation study and performance analysis
print("Ablation study: Remove reranking and compare retrieval accuracy.")
# ... implement ablation logic as needed ...

Ablation study: Remove reranking and compare retrieval accuracy.


### Improvement Proposals
1. Integrate a more powerful open-source LLM (e.g., Llama 3) for better answer generation.
2. Use advanced table extraction (e.g., Deep Learning-based PDF parsers) for more accurate structured data handling.
